In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [19]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 10), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating a4e19c9..283c440
Fast-forward
 augment_train.csv    | 16410 ++++++++++++++++++++++----------------------
 bertEUCL.ipynb       |   198 +-
 evaluation.csv       | 18062 ++++++++++++++++++++++++-------------------------
 preprocess.ipynb     |  2414 +++----
 useEUCL.ipynb        |   398 +-
 useTripleLoss .ipynb |   677 +-
 6 files changed, 19388 insertions(+), 18771 deletions(-)


In [4]:
import pandas as pd

In [20]:
df = pd.read_csv("augment_train.csv")
df_test = pd.read_csv("evaluation.csv")

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=4f2a47e570145780a39904a99fd61e11889c102cae6f2cff0fddf42c0450145c
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [7]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [8]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

In [9]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [10]:
def encoder1(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dropout(0.3)(embedding_output)
    a = tfl.Dense(128, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    outputs = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

def encoder2(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dropout(0.3)(embedding_output)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    outputs = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

In [11]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    text_embedd = encoder1("textUSEencoder")(inputText)

    #reasonEncoder
    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = encoder2("reasonUSEencoder")(inputReason)

    combined_features = tfl.concatenate([text_embedd, reason_embedd], name = 'merge_features')
    print(combined_features.shape)
    combined_features = tfl.Dropout(0.2)(combined_features)
    combined_features = tfl.Dense(16, activation = 'linear', kernel_regularizer=l2(1e-3))(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(4, activation = 'linear', kernel_regularizer=l2(1e-3))(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(1, activation = 'sigmoid')(combined_features)
    
    model = Model(inputs = [inputText, inputReason], outputs = [combined_features], name="uceModel")
    return model

In [22]:
model = get_model()

(None, 128)


In [23]:
model.summary()

Model: "uceModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 model_2 (Functional)           (None, 64)           256872768   ['input_5[0][0]']                
                                                                                                  
 model_3 (Functional)           (None, 64)           256831168   ['input_7[0][0]']                
                                                                                           

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss = "binary_crossentropy", metrics = [tf.metrics.BinaryAccuracy()])

In [25]:
loss_history = model.fit(siam_gen([df["clean_text"], df["reason"]], 128), 
                         validation_data = siam_gen([df_test["clean_text"], df_test["reason"]], 128),
                         steps_per_epoch=50, validation_steps=10, epochs = 50, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/50
50/50 [==============================] - 24s 329ms/step - loss: 1.1652 - binary_accuracy: 0.4986 - val_loss: 1.1158 - val_binary_accuracy: 0.5000
Epoch 2/50
50/50 [==============================] - 16s 315ms/step - loss: 1.1353 - binary_accuracy: 0.5005 - val_loss: 1.1032 - val_binary_accuracy: 0.5000
Epoch 3/50
50/50 [==============================] - 14s 289ms/step - loss: 1.1096 - binary_accuracy: 0.5044 - val_loss: 1.0919 - val_binary_accuracy: 0.5000
Epoch 4/50
50/50 [==============================] - 14s 287ms/step - loss: 1.0840 - binary_accuracy: 0.5148 - val_loss: 1.0755 - val_binary_accuracy: 0.5000
Epoch 5/50
50/50 [==============================] - 14s 288ms/step - loss: 1.0635 - binary_accuracy: 0.5120 - val_loss: 1.0568 - val_binary_accuracy: 0.5000
Epoch 6/50
50/50 [==============================] - 14s 288ms/step - loss: 1.0455 - binary_accuracy: 0.5173 - val_loss: 1.0390 - val_binary_accuracy: 0.5000
Epoch 7/50
50/50 [==============================] - 16s 31

In [26]:
from sklearn.metrics import brier_score_loss as brier_loss
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

In [27]:
y_preds = model.predict([df['clean_text'], df['reason']])
accuracy(df['label'], y_preds)

0.942310037809489

In [28]:
y_hat = (y_preds >= 0.5).astype(int)

In [29]:
print("Brier Loss                       : ", brier_loss(df['label'], y_preds))
print("Precision Score                  : ", precision_score(df['label'], y_hat))
print("Recall Score                     : ", recall_score(df['label'], y_hat))
print("F1 Score                         : ", f1_score(df['label'], y_hat))

Brier Loss                       :  0.0549892452622003
Precision Score                  :  1.0
Recall Score                     :  0.942310037809489
F1 Score                         :  0.9702982731554161


In [30]:
y_preds = model.predict([df_test['clean_text'], df_test['reason']])
accuracy(df_test['label'], y_preds)

0.5422222222222223

In [31]:
y_hat = (y_preds >= 0.5).astype(int)

In [32]:
print("Brier Loss                       : ", brier_loss(df_test['label'], y_preds))
print("Precision Score                  : ", precision_score(df_test['label'], y_hat))
print("Recall Score                     : ", recall_score(df_test['label'], y_hat))
print("F1 Score                         : ", f1_score(df_test['label'], y_hat))
print("Roc AUC Score                    : ", roc_auc_score(df_test['label'], y_preds))
print("BCE Loss                         : ", log_loss(df_test['label'], y_preds))



Brier Loss                       :  0.3241015325028808
Precision Score                  :  0.39792008757526
Recall Score                     :  0.7267577474175275
F1 Score                         :  0.5142655034190049
Roc AUC Score                    :  0.616703833622387
BCE Loss                         :  0.9542487061606952


In [ ]:
Brier Loss                       :  0.3203359289886762
Precision Score                  :  0.39153871283894154
Recall Score                     :  0.7987337554148617
F1 Score                         :  0.5254850378165077
Roc AUC Score                    :  0.6452798225451215
BCE Loss                         :  0.9062959283337825
